In [1]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
# import sklearn
# import tensorflow as tf
# import keras
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

# Internal Imports
from wsae_lstm.utils import dictmap_load

In [2]:
dict_dataframes_index = dictmap_load(path = "../data/processed/clean_data_index.xlsx")   
dict_dataframes_future = dictmap_load(path = "../data/processed/clean_data_future.xlsx")   

In [3]:
dict_dataframes_index.keys()

odict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])

In [4]:
dict_dataframes_index['csi300 index data'].head(2)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
0,2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
1,2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08


In [5]:
#df = dict_dataframes_index['csi300 index data']
#df = df.set_index(df['date'])

In [6]:
# df_tvt1 = df[(df['date'] >= '2008-7-01') & (df['date'] <= '2011-01-01')]
# df_tvt2 = df[(df['date'] >= '2008-10-01') & (df['date'] <= '2011-04-01')]
# df_tvt3 = df[(df['date'] >= '2009-01-01') & (df['date'] <= '2011-07-01')]

In [7]:
#df['date'][0] 

In [8]:
#df['date'][0]+monthdelta(30) 

In [9]:
def interval_split(df):
    dict_dataframes = {}
    split_count = 24
    month_increment = 0
    interval_index = 1
    
    df = df.set_index(df['date'])
    while split_count > 0:
        front = df['date'][0] 
        front = front + monthdelta(month_increment)
        back = df['date'][0]+monthdelta(30) 
        back = back + monthdelta(month_increment)
        month_increment += 3
        split_count -= 1
        df_interval = pd.DataFrame(df[(df['date'] >= front) & (df['date'] <= back)])
        dict_dataframes[interval_index] = df_interval
        interval_index += 1
        #print(front,back)
    return dict_dataframes

def dict_interval_split(dict_dataframes):
    subdict_dataframes = {}
    for dataframe in dict_dataframes:
        #print(dataframe)
        subdict_dataframes[dataframe] = interval_split(dict_dataframes[dataframe])
    return subdict_dataframes

In [10]:
dict_df_interval = dict_interval_split(dict_dataframes_index)

In [11]:
dict_df_interval.keys()

dict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])

In [12]:
dict_df_interval['csi300 index data'][1].head(2)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2008-07-01,2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08


In [13]:
dict_df_interval['csi300 index data'][24].head(2)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2014-04-01,2014-04-01,2143.72,2170.10,2141.15,2163.12,488664.03,80.08,2.866,-11.448999,45.062472,28.944,2150.219000,2158.446226,2150.5768,-13.439,2157.6278,40.279,1.897415,0.019530,7.622596e+07
2014-04-02,2014-04-02,2166.07,2182.68,2164.01,2180.73,650392.57,80.23,2.950,-8.435674,93.409788,19.568,2151.096615,2160.568204,2156.5625,6.287,2159.5638,37.689,1.758672,0.042995,1.775670e+08


In [14]:
type(dict_df_interval)

dict

In [15]:
dict_df_interval.keys()

dict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])

In [28]:
filename = "../data/pickled/clean_data_index_interval"
outfile = open(filename,'wb')
pickle.dump(dict_df_interval,outfile)
outfile.close()

In [29]:
infile = open("../data/pickled/clean_data_index_interval",'rb')
new_dict = pickle.load(infile)
infile.close()

In [25]:
len(new_dict)

6

In [26]:
new_dict['csi300 index data'][1].head()

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2008-07-01,2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [27]:
#dict_dataframes.keys()

In [100]:
#frames_to_excel(dict_dataframes,"../data/processed/clean_data_index_interval.xlsx")

In [101]:
#dict_dataframes = interval_split(dict_dataframes_index['csi300 index data'])

In [102]:
#dict_dataframes.keys()

In [103]:
#dict_dataframes[1].head(2)

In [104]:
# dict_dataframes_index['csi300 index data'].drop(['date'],axis=1,inplace=True)
# dict_dataframes_index['csi300 index data'].values

In [105]:
# validating
#df_train['date'][0]+monthdelta(3)

In [106]:
# testing
#df_train['date'][-1]+monthdelta(6)

In [107]:
# dict_dataframes_index['csi300 index data'].drop(['date'],axis=1,inplace=True)
# dict_dataframes_index['csi300 index data'].values

In [108]:
# from sklearn.model_selection import TimeSeriesSplit
# X = dict_dataframes_index['csi300 index data'].values
# tscv = TimeSeriesSplit(n_splits=6)
# for train_index, test_index in tscv.split(X):
#     train = X[train_index]
#     test = X[test_index]
#     print('Observations: %d' % (len(train) + len(test)))
#     print('Training Observations: %d' % (len(train)))
#     print('Testing Observations: %d' % (len(test)))